In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
location = "C:/Users/workspace/brad_public_workspace_on_win/SH_tongliang/data/"
file_name = "11_13_ML_BTC_daily_df_use_this.csv"

rnn_unit = 10  # 隐层数量
input_size = 7
output_size = 1
lr = 0.0006  # 学习率
#——————————————————导入数据——————————————————————
f = open(location+file_name)
df = pd.read_csv(f)  # 读入股票数据
data = df.iloc[:, 2:10].values  # 取第3-10列
range_ = 10
# print (len(data))

# 获取训练集，train_end由原来的5800改成了500.batch_size改成了10
def get_train_data(batch_size=10, time_step=20, train_begin=0, train_end=500):
    batch_index = []
    data_train = data[train_begin:train_end]
    normalized_train_data = (
        data_train - np.mean(data_train, axis=0)) / np.std(data_train, axis=0)  # 标准化
    train_x, train_y = [], []  # 训练集
    for i in range(len(normalized_train_data) - time_step):
        if i % batch_size == 0:
            batch_index.append(i)
        x = normalized_train_data[i:i + time_step, :7]
        y = normalized_train_data[i:i + time_step, 7, np.newaxis]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    batch_index.append((len(normalized_train_data) - time_step))
    # print ("train_x: ", train_x)# Brad新加, train_x data有数值
    # print ("train_y: ", train_y)# Brad新加，train_y data有数值
    return batch_index, train_x, train_y


# 获取测试集，test_begin改成了100
def get_test_data(time_step=20, test_begin=500):
    data_test = data[test_begin:1000]
    print ("data_test: ", data_test)
    mean = np.mean(data_test, axis=0)
    std = np.std(data_test, axis=0)
    # print ("mean: ", mean)# Brad新加的,all nan
    # print ("std", std)# Brad新加的. all nan
    normalized_test_data = (data_test - mean) / std  # 标准化
    # print ("normalized_test_data: ", normalized_test_data) # Brad新加的, all nan
    size = (len(normalized_test_data) + time_step -
            1) // time_step  # 有size个sample
    test_x, test_y = [], []
    for i in range(size - 1):
        x = normalized_test_data[i * time_step:(i + 1) * time_step, :7]
        y = normalized_test_data[i * time_step:(i + 1) * time_step, 7]
        test_x.append(x.tolist())
        test_y.extend(y)
    test_x.append((normalized_test_data[(i + 1) * time_step:, :7]).tolist())
    test_y.extend((normalized_test_data[(i + 1) * time_step:, 7]).tolist())
    # print ("test_x: ", test_x) # Brad新加，都是nan
    # print ("test_y", test_y) # Brad新加，都是nan
    return mean, std, test_x, test_y


#——————————————————定义神经网络变量——————————————————
# 输入层、输出层权重、偏置

weights = {
    'in': tf.Variable(tf.random_normal([input_size, rnn_unit])),
    'out': tf.Variable(tf.random_normal([rnn_unit, 1]))
}
biases = {
    'in': tf.Variable(tf.constant(0.1, shape=[rnn_unit, ])),
    'out': tf.Variable(tf.constant(0.1, shape=[1, ]))
}

#——————————————————定义神经网络变量——————————————————


def lstm(X):

    batch_size = tf.shape(X)[0]
    time_step = tf.shape(X)[1]
    w_in = weights['in']
    b_in = biases['in']
    input = tf.reshape(X, [-1, input_size])  # 需要将tensor转成2维进行计算，计算后的结果作为隐藏层的输入
    input_rnn = tf.matmul(input, w_in) + b_in
    # 将tensor转成3维，作为lstm cell的输入
    input_rnn = tf.reshape(input_rnn, [-1, time_step, rnn_unit])
    cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
    init_state = cell.zero_state(batch_size, dtype=tf.float32)
    output_rnn, final_states = tf.nn.dynamic_rnn(
        cell, input_rnn, initial_state=init_state, dtype=tf.float32)
    output = tf.reshape(output_rnn, [-1, rnn_unit])
    w_out = weights['out']
    b_out = biases['out']
    pred = tf.matmul(output, w_out) + b_out
    return pred, final_states

#————————————————训练模型————————————————————

# train_begin 从2000改成了100
def train_lstm(batch_size=60, time_step=20, train_begin=0, train_end=1100):
    X = tf.placeholder(tf.float32, shape=[None, time_step, input_size])
    Y = tf.placeholder(tf.float32, shape=[None, time_step, output_size])
    batch_index, train_x, train_y = get_train_data(
        batch_size, time_step, train_begin, train_end)
    with tf.variable_scope("sec_lstm"):
        pred, _ = lstm(X)
    loss = tf.reduce_mean(
        tf.square(tf.reshape(pred, [-1]) - tf.reshape(Y, [-1])))
    train_op = tf.train.AdamOptimizer(lr).minimize(loss)
    saver = tf.train.Saver(tf.global_variables(), max_to_keep=15)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(range_):  # 这个迭代次数，可以更改，越大预测效果会更好，但需要更长时间
            for step in range(len(batch_index) - 1):
                _, loss_ = sess.run([train_op, loss], feed_dict={X: train_x[batch_index[
                                    step]:batch_index[step + 1]], Y: train_y[batch_index[step]:batch_index[step + 1]]})
            print("Number of iterations:", i, " loss:", loss_)
        print("model_save5: ", saver.save(sess, 'model_save5\\modle.ckpt'))
        # 我是在window下跑的，这个地址是存放模型的地方，模型参数文件名为modle.ckpt
        # 在Linux下面用 'model_save2/modle.ckpt'
        print("The train has finished")
train_lstm()

#————————————————预测模型————————————————————


def prediction(time_step=20):
    X = tf.placeholder(tf.float32, shape=[None, time_step, input_size])
    mean, std, test_x, test_y = get_test_data(time_step)
    with tf.variable_scope("sec_lstm", reuse=True):
        pred, _ = lstm(X)
    saver = tf.train.Saver(tf.global_variables())
    with tf.Session() as sess:
        # 参数恢复
        module_file = tf.train.latest_checkpoint('model_save5')
        saver.restore(sess, module_file)
        test_predict = []
        # print ("test_x",test_x) # Brad新加的,也全是nan
        for step in range(len(test_x) - 1):
            prob = sess.run(pred, feed_dict={X: [test_x[step]]})
            predict = prob.reshape((-1))
            test_predict.extend(predict)
        test_y = np.array(test_y) * std[7] + mean[7]
        test_predict = np.array(test_predict) * std[7] + mean[7]
        acc = np.average(np.abs(
            test_predict - test_y[:len(test_predict)]) / test_y[:len(test_predict)])  # 偏差程度
        # print ("test_predict: ", test_predict)# Brad新加的, nan
        # print ("test_y: ", test_y)# Brad新加的,nan
#         print("The accuracy of this predict:", acc)
        # 以折线图表示结果
#         plt.figure()
#         plt.plot(list(range(len(test_predict))), test_predict, color='b',)
#         plt.plot(list(range(len(test_y))), test_y,  color='r')
#         plt.show()
        
prediction()

# def execute(rerun_times):
#     for i in range(rerun_times):
#         print (i)
#         if i <= rerun_times:
#             train_lstm()
#             prediction()
#         else:
#             break

In [ ]:
# execute(5)

# Predict and record prediction rate

In [36]:
# coding=utf-8

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime

%matplotlib inline

rnn_unit = 10  # 隐藏层单元数量
input_size = 4
output_size = 1
lr = 0.0005  # 学习率
location  = "C:/Users/workspace/brad_public_workspace_on_win/SH_tongliang/data/数据库/火币日线/"
file = "IF_cleaned_data_complete_version.csv"
# ——————————————————导入数据——————————————————————
df = pd.read_csv(location+file, engine="python")  # 读入IF期货数据
data = df.iloc[:, [1, 2, 3, 4]].values  # numpy.ndarray, 几个特征量：开盘价、最高价、最低价、收盘价
label = []
for i in range(1, len(data)):
    label.append(round(data[i, 3] - data[i - 1, 3], 4))

df.loc[1:, 'label'] = label  # 标签，以收盘价差作为涨跌,含幅度变化
date = datetime.now().strftime('%Y-%m-%d')


# 获取训练集
def get_train_data(batch_size=80,
                   time_step=20,
                   train_begin=0,
                   train_end=120000):
    batch_index = []
    data_train = data[train_begin:train_end]
    normalized_train_data = (
        data_train - np.mean(data_train, axis=0)) / np.std(
            data_train, axis=0)  # 标准化
    # print('type(normalized_train_data): ', type(normalized_train_data))
    # print('normalized_train_data.shape: ', normalized_train_data.shape)
    # normalized_train_data.shape: (5800, 8)
    train_x, train_y = [], []  # 训练集
    print (normalized_train_data)
    for i in range(len(normalized_train_data) - time_step):
        if i % batch_size == 0:
            batch_index.append(i)
        # x.shape: (20, 7)
        x = normalized_train_data[i:i + time_step, :3]
        # y.shape: (20, 1)
        y = normalized_train_data[i:i + time_step, 4, np.newaxis] 
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    # print('train_data x.shape: ', x.shape)
    # train_data x.shape:  (20, 6)
    # print('train_data y.shape: ', y.shape)
    # train_data y.shape:  (20, 1)
    # print('train_data y: ', y)
    batch_index.append((len(normalized_train_data) - time_step))
    return batch_index, train_x, train_y


# 获取测试集
def get_test_data(time_step=20, test_begin=120000):
    data_test = data[test_begin:]
    # print('data_test: ', data_test)
    test_y = data_test[:, 4]
    # print('test_y4: ', test_y)
    # data_train = data[:test_begin]
    mean = np.mean(data_test, axis=0)  # 改用已知的训练数据，避免用到未来数据
    std = np.std(data_test, axis=0)  # 同上
    normalized_test_data = (data_test - mean) / std  # 标准化
    size = (
        len(normalized_test_data) + time_step - 1) // time_step  # 有size个sample
    test_x, test_y = [], []
    for i in range(size - 1):
        x = normalized_test_data[i * time_step:(i + 1) * time_step, :3]
        y = normalized_test_data[i * time_step:(i + 1) * time_step, 4]
        test_x.append(x.tolist())
        test_y.extend(y)
    # print('type(x): ', type(x))
    # print('test_data x.shape: ', x.shape)
    # print('test_data y.shape: ', y.shape)
    # print('test_y3: ', test_y)
    test_x.append((normalized_test_data[(i + 1) * time_step:, :3]).tolist())
    test_y.extend((normalized_test_data[(i + 1) * time_step:, 4]).tolist())
    return mean, std, test_x, test_y


# ——————————————————定义神经网络变量——————————————————
# 输入层、输出层的权重、偏置

weights = {
    'in': tf.Variable(tf.random_normal([input_size, rnn_unit])),
    'out': tf.Variable(tf.random_normal([rnn_unit, 1]))
}
biases = {
    'in': tf.Variable(tf.constant(0.1, shape=[
        rnn_unit,
    ])),
    'out': tf.Variable(tf.constant(0.1, shape=[
        1,
    ]))
}


# ——————————————————定义神经网络变量——————————————————
def lstm(X):

    batch_size = tf.shape(X)[0]
    time_step = tf.shape(X)[1]
    w_in = weights['in']
    b_in = biases['in']
    input = tf.reshape(X, [-1, input_size])  # 需要将tensor转成2维进行计算，计算后的结果作为隐藏层的输入
    input_rnn = tf.matmul(input, w_in) + b_in
    input_rnn = tf.reshape(
        input_rnn, [-1, time_step, rnn_unit])  # 将tensor转成3维，作为lstm cell的输入
    cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
    init_state = cell.zero_state(batch_size, dtype=tf.float32)
    output_rnn, final_states = tf.nn.dynamic_rnn(
        cell, input_rnn, initial_state=init_state, dtype=tf.float32)
    output = tf.reshape(output_rnn, [-1, rnn_unit])
    w_out = weights['out']
    b_out = biases['out']
    pred = tf.matmul(output, w_out) + b_out
    return pred, final_states


# ————————————————训练模型————————————————————


def train_lstm(batch_size=80, time_step=20, train_begin=0, train_end=120000):
    X = tf.placeholder(tf.float32, shape=[None, time_step, input_size])
    Y = tf.placeholder(tf.float32, shape=[None, time_step, output_size])
    batch_index, train_x, train_y = get_train_data(batch_size, time_step,
                                                   train_begin, train_end)
    with tf.variable_scope("sec_lstm"):
        pred, _ = lstm(X)
    loss = tf.reduce_mean(
        tf.square(tf.reshape(pred, [-1]) - tf.reshape(Y, [-1])))
    train_op = tf.train.AdamOptimizer(lr).minimize(loss)
    saver = tf.train.Saver(tf.global_variables(), max_to_keep=15)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(200):  # 这个迭代次数，可以更改，越大预测效果会更好，但需要更长时间
            for step in range(len(batch_index) - 1):
                _, loss_ = sess.run(
                    [train_op, loss],
                    feed_dict={
                        X: train_x[batch_index[step]:batch_index[step + 1]],
                        Y: train_y[batch_index[step]:batch_index[step + 1]]
                    })
            print("Number of iterations:", i, " loss:", loss_)
            with open( location+ 'loss_' + date + '.csv', 'w') as f:
                f.write(str(loss_) + '\n')
        # print("model_save: ", saver.save(sess, 'model_save2\\modle.ckpt'))
        print("model_save: ", saver.save(sess, location+'model_save5/modle.ckpt'))
        # 我是在window下跑的，这个地址是存放模型的地方，模型参数文件名为modle.ckpt
        # 在Linux下面用 'model_save2/modle.ckpt'
        print("The train has finished")


train_lstm()


# ————————————————预测模型————————————————————
def prediction(time_step=20):
    X = tf.placeholder(tf.float32, shape=[None, time_step, input_size])
    mean, std, test_x, test_y = get_test_data(time_step)
    with tf.variable_scope("sec_lstm", reuse=True):
        pred, _ = lstm(X)
    saver = tf.train.Saver(tf.global_variables())
    with tf.Session() as sess:
        # 参数恢复
        module_file = tf.train.latest_checkpoint('model_save5')
        saver.restore(sess, module_file)
        test_predict = []
        for step in range(len(test_x) - 1):
            # prob.shape: (20, 1)
            prob = sess.run(pred, feed_dict={X: [test_x[step]]})
            # print('prob.shape: ', prob.shape)
            # predict.shape: (20, )
            predict = prob.reshape((-1))
            test_predict.extend(predict)
        # print('test_y1: ', test_y)
        test_y = np.array(test_y) * std[4] + mean[4]
        # print('test_y2: ', test_y)
        # print('test_y.shape: ', test_y.shape)
        test_predict = np.array(test_predict) * std[4] + mean[4]
        acc = np.average(
            np.abs(test_predict - test_y[:len(test_predict)]) /
            test_y[:len(test_predict)])  # 偏差程度
        print("The accuracy of this predicet:", 1 - acc)
        with open(location+'test_y_' + date + '.csv', 'w') as f:
            f.write(str(test_y) + '\n')

        with open(location+'test_predict_' + date + '.csv', 'w') as f:
            f.write(str(test_predict))

        with open(location+'accuracy_' + date + '.csv', 'w') as f:
            f.write(str(1-acc) + '\n')
        # 以折线图表示结果
        # print('len(test_predict): ', len(test_predict))
        # print('len(test_y): ', len(test_y))
        # print('len(data): ', len(data))
        plt.figure()
        # fig = plt.subplot(111)
        plt.plot(
            list(range(len(test_predict))),
            test_predict,
            color='blue',
            label='predict',
        )
        plt.plot(list(range(len(test_y))), test_y, color='red', label='real')
        # plt.plot(
        #     list(range(len(data))), data, color='r', label='real_all')
        plt.legend(loc='upper left')
        plt.ylabel('high')
        plt.show()


prediction()
# The accuracy of this predict: 0.016965209201494337


[[ 0.78397108  0.80333841  0.79002065  0.8092925 ]
 [ 0.80925965  0.806143    0.81008078  0.8064826 ]
 [ 0.80680104  0.80158555  0.79846702  0.79418927]
 ...
 [-0.20122939 -0.20315827 -0.19750071 -0.20086774]
 [-0.20052693 -0.20280769 -0.19714878 -0.19911155]
 [-0.19877078 -0.20280769 -0.19750071 -0.20086774]]


IndexError: index 4 is out of bounds for axis 1 with size 4

In [ ]:
y = normalized_train_data[i:i + time_step, 4, np.newaxis] 

In [9]:
x1 = np.array([1, 2, 3, 4], [2, 3, 4, 4])
# x2 = np.array([5, 4, 3])

TypeError: data type not understood

In [15]:
x = np.array([[1, 3, 2, 6], [2, 3, 5, 1], 
          [3, 2, 5, 1], [2, 4, 1, 6]])

In [16]:
x

array([[1, 3, 2, 6],
       [2, 3, 5, 1],
       [3, 2, 5, 1],
       [2, 4, 1, 6]])

In [ ]:
        x = normalized_train_data[i:i + time_step, :3]
        # y.shape: (20, 1)
        y = normalized_train_data[i:i + time_step, 4, np.newaxis] 

In [24]:
x[0:3, :3]

array([[1, 3, 2],
       [2, 3, 5],
       [3, 2, 5]])

In [28]:
x[0:3, 3, np.newaxis]

array([[6],
       [1],
       [1]])

# 正在进行的米筐机器学习算法代码：

In [35]:
# 可以自己import我们平台支持的第三方python模块，比如pandas、numpy等。
import pandas as pd
import numpy as np
from datetime import timedelta
from pybrain.datasets import SequentialDataSet
from pybrain.tools.shortcuts import buildNetwork
from pybrain.structure.networks import Network
from pybrain.structure.modules import ReluLayer, LSTMLayer
from pybrain.supervised import RPropMinusTrainer

# 训练trainX和trainY，并返回神经网络net
def train(context, trainX, trainY):
    ds = SequentialDataSet(4, 1)
    for dataX, dataY in zip(trainX, trainY):
        ds.addSample(dataX, dataY)
    net = buildNetwork(4, 1, 1, hiddenclass=LSTMLayer, outputbias=False, recurrent=True)
    trainer = RPropMinusTrainer(net, dataset=ds)
    EPOCHS_PER_CYCLE = 5
    CYCLES = 5
    for i in range(CYCLES):
        trainer.trainEpochs(EPOCHS_PER_CYCLE)
    return net, trainer.testOnData()

# 更新数据集data
def load(context, ticker):
    close = history(90, '1d', 'close')[ticker]
    high  = history(90, '1d', 'high')[ticker]
    low   = history(90, '1d', 'low')[ticker]
    volume= history(90, '1d', 'volume')[ticker]
    data = pd.DataFrame({'close':close.values,
                         'high':high.values,
                         'low':low.values,
                         'volume':volume.values},index=close.index)
    # print (data)
    context.position_ratio.append([data['close'].mean(),
                                   data['high'].mean(), 
                                   data['low'].mean(), 
                                   data['volume'].mean()])
    context.shape_ratio.append([data['close'].std(),
                                data['high'].std(),
                                data['low'].std(),
                                data['volume'].std()])
    data['close']  = (data['close'] - context.position_ratio[-1][0]) / context.shape_ratio[-1][0]
    data['high']   = (data['high'] - context.position_ratio[-1][1]) / context.shape_ratio[-1][1]
    data['low']    = (data['low'] - context.position_ratio[-1][2]) / context.shape_ratio[-1][2]
    data['volume'] = (data['volume'] - context.position_ratio[-1][3]) / context.shape_ratio[-1][3]
    
    return data

# 剔除情况特殊的黑名单股，只看策略效果，排除个体问题
def filter_blacklist(context, stock_list):
    return [ticker for ticker in stock_list if ticker not in context.blacklist]
    
def filter_stlist(stock_list):
    return [ticker for ticker in stock_list if not is_st_stock(ticker)]

# 建模，每3个月运行一次，用过去6个月训练
def modelize(context, bar_dict):
    if context.every_3_months % 3 != 0: 
        context.every_3_months += 1
        return 0
    print('-'*65)
    print('------'+'{:-^59}'.format('modelizing'))
    context.position_ratio = []
    context.shape_ratio    = []
    context.data  = []
    context.net   = []
    context.list  = []
    templist = list(get_fundamentals(query(fundamentals.eod_derivative_indicator.market_cap)
                                    .order_by(fundamentals.eod_derivative_indicator.market_cap.asc())
                                    .limit(context.num*5)).columns)
    context.list = filter_blacklist(context, filter_stlist(templist))[:context.num]
    names  = []
    scores = []
    for ticker in context.list:
        names.append('{:<11}'.format(ticker))
        data = load(context, ticker)
        trainX = data.ix[:-1,:].values
        trainY = data.ix[1:,0].values
        net, mse = train(context, trainX, trainY)
        context.data.append(data)
        context.net.append(net)
        scores.append('{:<11}'.format(str(mse)[:6]))
        if np.isnan(mse):
            context.blacklist.append(ticker)
            context.mflag = 0
            return 0
    context.pct = [0] * context.num
    print('------'+'{:-^59}'.format('finished'))
    print('-'*65)
    print(' nm | '+' '.join(names))
    print('mse | '+' '.join(scores))

    context.mflag = 1  # 标记已经建模
    context.tflag = 0
    context.every_3_months += 1

def mkt_panic():
    # 连续两天大盘跌破3个点，或者大盘跌破5个点
    mkt = history(3, '1d', 'close')['000001.XSHG']
    panic = (mkt[-1]/mkt[-2] < 0.97 and mkt[-2]/mkt[-3] < 0.97) or mkt[-1]/mkt[-2] < 0.95
    if panic:
        print('!!!!!!'+'{:!^59}'.format('panic'))
        return 1
    return 0

# 最后利用每3个月更新的模型，每天进行交易，预测涨幅超过a就买入，预测跌幅超过b则卖出
def trade(context,bar_dict):
    
    while context.mflag == 0: modelize(context, bar_dict)
    
    trash_bin = [ticker for ticker in context.portfolio.positions if ticker not in context.list]
    for ticker in trash_bin: order_target_percent(ticker, 0)
    
    actual_close  = []
    actual_high   = []
    actual_low    = []
    actual_vol    = []
    actual_open   = []
    actual_data   = []
    predict_close = []
    
    for i in range(context.num):
        actual_close.append((history(1,'1d','close')[context.list[i]][0] - context.position_ratio[i][0]) / context.shape_ratio[i][0])
        actual_high.append((history(1,'1d','high')[context.list[i]][0] - context.position_ratio[i][1]) / context.shape_ratio[i][1])
        actual_low.append((history(1,'1d','low')[context.list[i]][0] - context.position_ratio[i][2]) / context.shape_ratio[i][2])
        actual_vol.append((history(1,'1d','volume')[context.list[i]][0] - context.position_ratio[i][3]) / context.shape_ratio[i][3])
        actual_open.append((history(1,'1m','close')[context.list[i]][0] - context.position_ratio[i][0]) / context.shape_ratio[i][0])
        actual_data.append([actual_close[i],actual_high[i],actual_low[i],actual_vol[i]])
        predict_close.append(context.net[i].activate(actual_data[i])[0])

    if context.tflag == 0: 
        context.temp_pc = predict_close
    
    r = [float((pc*shape_ratio[0]+position_ratio[0]) / (ao*shape_ratio[0]+position_ratio[0]) - 1) for pc, ao, shape_ratio, position_ratio in zip(predict_close, actual_open, context.shape_ratio, context.position_ratio)]
    
    temp_r = [float((pc*shape_ratio[0]+position_ratio[0]) / (tpc*shape_ratio[0]+position_ratio[0]) - 1) for pc, tpc, shape_ratio, position_ratio in zip(predict_close, context.temp_pc, context.shape_ratio, context.position_ratio)]
    
    # The essence of this strategy
    hybrid_r = [max(ri,temp_ri,ri+temp_ri) for ri, temp_ri in zip(r,temp_r)]
    bad_hybrid_signal = sum([x <= 0 for x in hybrid_r])
    a, b = 0.00, -0.01
    panic = mkt_panic()
    for i in range(context.num):
        if panic or 0 < context.post_panic < 22 * context.num:
            context.pct[i] = 0
            context.post_panic = (1 - panic) * (context.post_panic + 1) + panic
        elif hybrid_r[i] > a:
            context.pct[i] = min(context.pct[i] + .5/context.num, 2/context.num)
            context.post_panic = 0
        elif hybrid_r[i] < b or bad_hybrid_signal > 3*context.num//5:
            context.pct[i] = max(context.pct[i] - .5/context.num, 0)
            context.post_panic = 0
        

    if context.tflag == 1: print(' ac | '+' '.join(['{:<11}'.format(str(ac)[:6]) for ac in actual_close]))
    print('-'*65)
    print(' ao | '+' '.join(['{:<11}'.format(str(ao)[:6]) for ao in actual_open]))
    print(' pc | '+' '.join(['{:<11}'.format(str(pc)[:6]) for pc in predict_close]))
    print('  r | '+' '.join(['{:<11}'.format(str(ri)[:6]) for ri in hybrid_r]))
    pct = sum([context.portfolio.positions[ticker].market_value for ticker in context.portfolio.positions])/(context.portfolio.market_value+context.portfolio.cash)
    tot_pct = max(sum(context.pct), 1)
    context.pct = list(map(lambda x: x/tot_pct, context.pct))
    print('  % | '+' '.join(['{:<11}'.format(str(p)[:6]) for p in context.pct]))
    plot('total position', pct * 100)
    for i in range(context.num): order_target_percent(context.list[i], context.pct[i])
    context.tflag = 1
    context.temp_pc = predict_close
    
# 在这个方法中编写任何的初始化逻辑。context对象将会在你的算法策略的任何方法之间做传递。
def init(context):
    context.temp_pc        = []
    context.every_3_months = 0
    context.tflag          = 0
    context.mflag          = 0
    context.position_ratio = []
    context.shape_ratio    = []
    context.num            = 20
    context.list           = []
    context.pct            = [0] * context.num
    context.net            = []
    context.data           = []
    context.post_panic     = 0
    context.blacklist      = [
                              '000004.XSHE','000546.XSHE',
                              '000594.XSHE','002352.XSHE',
                              '300176.XSHE','300260.XSHE',
                              '300372.XSHE','600137.XSHG',
                              '600306.XSHG','600656.XSHG',
                             ]
    scheduler.run_monthly(modelize,1)
    scheduler.run_daily(trade, time_rule=market_open(minute=1))

# before_trading此函数会在每天交易开始前被调用，当天只会被调用一次
def before_trading(context):
    pass

# 你选择的证券的数据更新将会触发此段逻辑，例如日或分钟历史数据切片或者是实时数据切片更新
def handle_bar(context, bar_dict):
    pass

ModuleNotFoundError: No module named 'structure'